In [1]:
#coding:utf-8
import torch as t
import numpy as np
from torch import nn
import ipdb

In [2]:
import sys
sys.path.append('D:\\Jupyter\\Python\\ATAE-LSTM')
import Ipynb_importer
from config import opt
from data.Embedding import emb
from models.BasicModule import BasicModule
from data.AspClas_ import AspClas

Embedding : have input words : 1
Embedding : have input words : 2
Embedding : have input words : 4
Embedding : have input words : 8
Embedding : have input words : 16
Embedding : have input words : 32
Embedding : have input words : 64
Embedding : have input words : 128
Embedding : have input words : 256
Embedding : have input words : 512
Embedding : have input words : 1024
Embedding : have input words : 2048
Embedding : have input words : 4096
Embedding : have input words : 8192
Embedding : have input words : 16384


In [3]:
class AE_GRU(BasicModule):
    def __init__(self):
        super(AE_GRU, self).__init__()
        
        self.embedding = emb._make_layer_()
        
        self.gru = nn.GRU(opt.hidden_size*2, opt.hidden_size, 1)
        self.h0 = t.randn(1, 1, opt.hidden_size)
        
        self.lin = nn.Linear(opt.hidden_size, opt.classes)
        self.tanh = t.nn.Tanh()
        self.softmax = nn.Softmax(dim=0)
        
        return
    
    def forward(self, x):
        # word representation
        w = x[0]
        N = len(w)
        # aspect term
        v = x[1]
        # assert len(v)==1  # use mean()
        
        # e.g.
        # w torch.Size([16])
        # v torch.Size([1])
        
        e1 = self.embedding(x[0])
        e2 = self.embedding(x[1]).mean(dim=0).view(-1).expand(e1.size())
        # e.g.
        # e1 torch.Size([16, 300])
        # e2 torch.Size([1, 300]) -> torch.Size([16, 300])
        
        wv = t.cat((e1.view(N,1,opt.hidden_size), e2.view(N,1,opt.hidden_size)), dim=-1)
        # e.g.
        # wv torch.Size([16, 1, 600])
        
        out, hn = self.gru(wv, self.h0)
        # e.g.
        # out torch.Size([16, 1, 300])
        # hn torch.Size([1, 1, 300])
        
        y = self.softmax(self.lin(hn.view(opt.hidden_size)))
        # e.g.
        # y torch.Size([3])
        
        return y

In [4]:
if __name__=='__main__':
    %pdb on
    testDataset = AspClas(opt.base_root+'/data/restaurants-trial.xml')
    model = AE_GRU()
    for text, aspect, sentiment in testDataset:
        x = (text, aspect)
        y = model(x)
        print(y)
    print(len(list(model.parameters())))

Automatic pdb calling has been turned ON
tensor([0.2366, 0.4165, 0.3469], grad_fn=<SoftmaxBackward>)
tensor([0.3710, 0.3484, 0.2806], grad_fn=<SoftmaxBackward>)
tensor([0.3815, 0.3369, 0.2816], grad_fn=<SoftmaxBackward>)
tensor([0.2799, 0.3123, 0.4078], grad_fn=<SoftmaxBackward>)
tensor([0.3329, 0.3325, 0.3347], grad_fn=<SoftmaxBackward>)
tensor([0.2787, 0.3304, 0.3909], grad_fn=<SoftmaxBackward>)
tensor([0.3025, 0.3696, 0.3279], grad_fn=<SoftmaxBackward>)
tensor([0.2767, 0.3343, 0.3890], grad_fn=<SoftmaxBackward>)
tensor([0.3337, 0.3564, 0.3099], grad_fn=<SoftmaxBackward>)
tensor([0.2754, 0.3672, 0.3574], grad_fn=<SoftmaxBackward>)
tensor([0.3447, 0.3318, 0.3236], grad_fn=<SoftmaxBackward>)
tensor([0.3055, 0.4136, 0.2809], grad_fn=<SoftmaxBackward>)
tensor([0.3304, 0.3402, 0.3294], grad_fn=<SoftmaxBackward>)
tensor([0.3205, 0.3747, 0.3048], grad_fn=<SoftmaxBackward>)
tensor([0.3359, 0.3548, 0.3093], grad_fn=<SoftmaxBackward>)
tensor([0.2808, 0.2990, 0.4202], grad_fn=<SoftmaxBackward>)